In [1]:
import time
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# 新增偽裝成chrome瀏覽器的標頭
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [3]:
# 透過 requests 套件發送 HTTP GET 請求，訪問網址 https://www.moneydj.com/z/zg/zge/zge_E_E.djhtm
response = requests.get("https://www.moneydj.com/z/zg/zge/zge_E_E.djhtm", headers=headers)

In [4]:
# 使用bs4從網頁內容中取得select標籤的內容，其XPATH為 //*[@id="oHeadTitle"]/select[1]
soup = BeautifulSoup(response.text, "html.parser")
select = soup.select("#oHeadTitle > select")[0]

# 取得所有的option標籤
options = select.find_all("option")

# 逐一取得每個option標籤的value和text
topic_list = []
for option in options:
    # print(option["value"], option.text)
    topic_list.append([option["value"], option.text])


In [5]:
# 將存放股票題材的清單轉換成DataFrame
topic_df = pd.DataFrame(topic_list, columns=["value", "text"])
print(topic_df.shape)
topic_df.head(3) 

(188, 2)


,value,text
0,EH000237,3D IC概念股
1,EH001253,3D全息投影概念股
2,EH000227,3D列印概念股


In [6]:
def _read_topic_stocks(tar_html):
    """ 讀取股票題材清單中的股票代號和名稱 """

    # 使用bs4從網頁內容中取得select標籤的內容，其XPATH為 //*[@id="oMainTable"]/tbody
    soup = BeautifulSoup(tar_html, "html.parser")

    # 以id為oMainTable取得table標籤
    table = soup.select("#oMainTable")[0]

    # 從第二個tr標籤開始取得
    trs = table.find_all("tr")[1:]

    # 逐一取得每個tr標籤的td標籤
    stock_list = []
    for tr in trs:
        # 逐一取得每個td標籤的內容
        tds = tr.find_all("td")
        for td in tds:
            # 第一個td標籤中有javasccript，讀取其script內容
            if td.find("script"):
                # 取得GenLink2stk函數呼叫的參數
                td_content = td.find("script").text.split("'")
                stock_no = td_content[1][2:]
                stock_name = td_content[3]
                stock_list.append([stock_no, stock_name])
    
    if len(stock_list) > 0:
        stock_df= pd.DataFrame(stock_list, columns=["stock_no", "stock_name"])
    else:
        stock_df = pd.DataFrame([], columns=["stock_no", "stock_name"])

    return stock_df

In [7]:
# 將topic_df中的value欄位值，建立一個迴圈遍歷每一個值，並將其帶入url_template中的{topic}，組成新的url
url_template = "https://www.moneydj.com/z/zg/zge/zge_{topic}_1.djhtm"
tar_html = ""
topic_stock_df = pd.DataFrame([], columns=["stock_no", "stock_name", "topic"])
for _, topic in topic_df.iterrows():
    print(f"topic_name: {topic['text']}")
    url = url_template.format(topic=topic["value"])

    # 透過 requests 套件發送 HTTP GET 請求，訪問網址 url
    response = requests.get(url, headers=headers)
    tar_html = response.text
    tmp_topic_stock_df =_read_topic_stocks(tar_html)
    if not tmp_topic_stock_df.empty:
        tmp_topic_stock_df["topic"] = topic["text"]
        topic_stock_df = pd.concat([topic_stock_df, tmp_topic_stock_df], ignore_index=True)

    print(tmp_topic_stock_df)
    print("\n\n")

    sleep_time = random.randint(1, 10)
    print(f"sleep_time: {sleep_time}")
    time.sleep(sleep_time)  # 隨機暫停1～10秒
    # break

topic_name: 3D IC概念股
  stock_no stock_name     topic
0     3711      日月光投控  3D IC概念股
1     3583         辛耘  3D IC概念股
2     6187         萬潤  3D IC概念股
3     6239         力成  3D IC概念股
4     6510         精測  3D IC概念股
5     2330        台積電  3D IC概念股
6     3374         精材  3D IC概念股



sleep_time: 5
topic_name: 3D全息投影概念股
  stock_no stock_name      topic
0     5371        中光電  3D全息投影概念股
1     2308        台達電  3D全息投影概念股
2     2412        中華電  3D全息投影概念股
3     2352        佳世達  3D全息投影概念股
4     3504        揚明光  3D全息投影概念股
5     4976         佳凌  3D全息投影概念股
6     2489         瑞軒  3D全息投影概念股



sleep_time: 10
topic_name: 3D列印概念股
   stock_no stock_name    topic
0      6123         上奇  3D列印概念股
1      2395         研華  3D列印概念股
2      9105     泰金寶-DR  3D列印概念股
3      8081         致新  3D列印概念股
4      2347         聯強  3D列印概念股
5      4129         聯合  3D列印概念股
6      1504         東元  3D列印概念股
7      4722        國精化  3D列印概念股
8      1785        光洋科  3D列印概念股
9      8416         實威  3D列印概念股
10     2373        震旦行  3D列印概念

In [8]:
# 將topic_stock_df輸出EXCEL檔案
topic_stock_df.to_excel("tw_stock_topics.xlsx", index=False)